In [8]:
!pip install openai
!pip install weaviate-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.3/94.3 kB 2.4 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.8/214.8 kB 6.4 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=292f05ec45692a69149704bacbdc902bcf93cee64373ee84ea0f70deaad23779
  Stored in directory: /root/.cache/pip/wheels/2d/55/25/123071088f4e466746cbadc923b1a31e08cea99ea9ef6bb35e
Successfully built validators


In [3]:
import weaviate
from weaviate.embedded import EmbeddedOptions

client = weaviate.Client(
  url="http://weaviate:8080"
  # embedded_options=EmbeddedOptions()
)

In [7]:
current_schema = client.schema.get()
for class_obj in current_schema['classes']:
    class_name = class_obj['class']
    client.schema.delete_class(class_name)

In [9]:
current_schema = client.schema.get()
current_schema

{'classes': []}

In [11]:
class_obj = {
    "class": "Test",
    "vectorizer": "none", # we are providing the vectors so this field is none
    "description": "holds all laws",
    "properties": [
        {
            "name": "content",
            "dataType": ["text"],
        },
        {
            "name": "description",
            "dataType": ["text"],
        },
    ]
}

client.schema.create_class(class_obj)

In [36]:
article_1 = {
    "content": "Example Article 1",
    "description": "This is an example article about machine learning and artificial intelligence.",
}

article_2 = {
    "content": "Example Article 2",
    "description": "This is another example article about natural language processing and its applications.",
}

In [45]:
with client.batch as batch:  # Context manager manages batch flushing
    batch.batch_size=1
    batch.dynamic=True
    for data_obj in [article_1, article_2]:
        batch.add_data_object(data_obj, "Test", vector=[-0.1, -0.2, -0.3, -0.4, -0.5])

In [46]:
client.query.aggregate("Test").with_meta_count().do()

{'data': {'Aggregate': {'Test': [{'meta': {'count': 12}}]}}}

In [47]:
search_vector = [0.6] * 5


results = client.query.get(
    "Test", ["content", "description"]
).with_near_vector(
    {"vector": search_vector}
).with_additional(
    ["distance", "id", "vector"]
).with_limit(10).do()  # We use a limit of 10 in the video - lowered here for brevity

In [48]:
results

{'data': {'Get': {'Test': [{'_additional': {'distance': 1.9045341,
      'id': '141c5f74-f4f8-4756-947c-816af9e1635e',
      'vector': [-0.1, -0.2, -0.3, -0.4, -0.5]},
     'content': 'Example Article 2',
     'description': 'This is another example article about natural language processing and its applications.'},
    {'_additional': {'distance': 1.9045341,
      'id': 'e9499360-4735-497b-9efd-e95ae7420f21',
      'vector': [-0.1, -0.2, -0.3, -0.4, -0.5]},
     'content': 'Example Article 1',
     'description': 'This is an example article about machine learning and artificial intelligence.'},
    {'_additional': {'distance': 1.9045341,
      'id': 'a4470b47-35eb-4efd-bfea-782b4a6b94fb',
      'vector': [-0.1, -0.2, -0.3, -0.4, -0.5]},
     'content': 'Example Article 2',
     'description': 'This is another example article about natural language processing and its applications.'},
    {'_additional': {'distance': 1.9045341,
      'id': 'afa199db-d243-45db-8cb7-2ca79a38e355',
      'v

In [14]:
client.batch.create_objects([article_1, article_2], ["Test", "Test"])

TypeError: Batch.create_objects() takes 1 positional argument but 3 were given

In [ ]:
data_obj = {
    "content": doc
}
client.data_object.create(
    data_obj,
    "Post",
    generate_uuid('Post',doc),
    vector= vec,
)

In [7]:
schema = client.schema.get()
print(schema)

{'classes': [{'class': 'Post', 'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2}, 'cleanupIntervalSeconds': 60, 'stopwords': {'additions': None, 'preset': 'en', 'removals': None}}, 'properties': [{'dataType': ['text'], 'name': 'content', 'tokenization': 'word'}], 'replicationConfig': {'factor': 1}, 'shardingConfig': {'virtualPerPhysical': 128, 'desiredCount': 1, 'actualCount': 1, 'desiredVirtualCount': 128, 'actualVirtualCount': 128, 'key': '_id', 'strategy': 'hash', 'function': 'murmur3'}, 'vectorIndexConfig': {'skip': False, 'cleanupIntervalSeconds': 300, 'maxConnections': 64, 'efConstruction': 128, 'ef': -1, 'dynamicEfMin': 100, 'dynamicEfMax': 500, 'dynamicEfFactor': 8, 'vectorCacheMaxObjects': 1000000000000, 'flatSearchCutoff': 40000, 'distance': 'cosine', 'pq': {'enabled': False, 'bitCompression': False, 'segments': 0, 'centroids': 256, 'encoder': {'type': 'kmeans', 'distribution': 'log-normal'}}}, 'vectorIndexType': 'hnsw', 'vectorizer': 'none'}]}


In [11]:
schema["classes"]

[{'class': 'Post',
  'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
   'cleanupIntervalSeconds': 60,
   'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
  'properties': [{'dataType': ['text'],
    'name': 'content',
    'tokenization': 'word'}],
  'replicationConfig': {'factor': 1},
  'shardingConfig': {'virtualPerPhysical': 128,
   'desiredCount': 1,
   'actualCount': 1,
   'desiredVirtualCount': 128,
   'actualVirtualCount': 128,
   'key': '_id',
   'strategy': 'hash',
   'function': 'murmur3'},
  'vectorIndexConfig': {'skip': False,
   'cleanupIntervalSeconds': 300,
   'maxConnections': 64,
   'efConstruction': 128,
   'ef': -1,
   'dynamicEfMin': 100,
   'dynamicEfMax': 500,
   'dynamicEfFactor': 8,
   'vectorCacheMaxObjects': 1000000000000,
   'flatSearchCutoff': 40000,
   'distance': 'cosine',
   'pq': {'enabled': False,
    'bitCompression': False,
    'segments': 0,
    'centroids': 256,
    'encoder': {'type': 'kmeans', 'distribution': 'log-normal'

In [2]:
import os

In [4]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
import os
import openai

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
      {"role": "user", "content":
           "I want you to become a Unix bash terminal scripting expert.\nOutput only the code needed to fulfill the task, add no explanations or descriptions of the involved procedure, including explantations before and after the code or script."
      },
      {"role": "user", "content": 'a script using wget that downloads a website and extract all links'}
  ],
  temperature=0,
  max_tokens=100,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  # stop=["\n"]
)

In [6]:
print(response["choices"][0]["message"]["content"])

wget --recursive --no-parent --spider -R "index.html*" -nd -e robots=off <website URL> 2>&1 | grep -o -E "http[s]?://[a-zA-Z0-9./?=_-]*" > links.txt


In [129]:
import requests
from bs4 import BeautifulSoup

def extract_text(soup, tag, class_name):
    element = soup.find(tag, class_=class_name)
    return element.get_text(strip=True) if element else ""

# Set your URL here
url = "https://rkrattsbaser.gov.se/sfst?bet=2015:632"

# Send an HTTP request and parse the HTML content
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Extract text from each part of the HTML structure
main_wrapper = extract_text(soup, "div", "main wrapper")
content = extract_text(soup, "div", "content")
search_results = extract_text(soup, "div", "search-results")
search_main = extract_text(soup, "div", "search-main")
search_results_content = extract_text(soup, "div", "search-results-content")

# Save the extracted text in a dictionary
result_dict = {
    "main_wrapper": main_wrapper,
    "content": content,
    "search_results": search_results,
    "search_main": search_main,
    "search_results_content": search_results_content
}

print(result_dict)

{'main_wrapper': 'FörfattningsregisterFörfattningar fulltextKommittédirektivKommittéregistretOmPost1av1träffarSökresultatFöregående·NästaSFS-nummer ·\r\n                    2015:632 ·Visa registerLag (2015:632) om talan om skattetillägg i vissa fallDepartement:Finansdepartementet S3Utfärdad:2015-11-12Ikraft:2016-01-01 överg.best.Inledande bestämmelser\r\n\r\n1 § Denna lag gäller om en felaktighet eller passivitet kan \r\nutgöra grund för såväl beslut om skattetillägg enligt \r\nskatteförfarandelagen (2011:1244) som åtal för brott enligt \r\nskattebrottslagen (1971:69) för samma fysiska person.\r\n\r\n2 § I lagen finns bestämmelser om förfarandet när en fråga om \r\nskattetillägg prövas i samma rättegång som ett åtal för \r\nskattebrott. Lagen innehåller också bestämmelser om \r\nförfarandet hos åklagaren samt vissa bestämmelser om \r\nverkställighet. \r\n\r\n3 § När en fråga om skattetillägg ska prövas enligt denna \r\nlag, tillämpas bestämmelserna om skattetillägg i 49 kap. och \r\n51

In [131]:
result_dict["search_results_content"]

'SFS-nummer ·\r\n                    2015:632 ·Visa registerLag (2015:632) om talan om skattetillägg i vissa fallDepartement:Finansdepartementet S3Utfärdad:2015-11-12Ikraft:2016-01-01 överg.best.Inledande bestämmelser\r\n\r\n1 § Denna lag gäller om en felaktighet eller passivitet kan \r\nutgöra grund för såväl beslut om skattetillägg enligt \r\nskatteförfarandelagen (2011:1244) som åtal för brott enligt \r\nskattebrottslagen (1971:69) för samma fysiska person.\r\n\r\n2 § I lagen finns bestämmelser om förfarandet när en fråga om \r\nskattetillägg prövas i samma rättegång som ett åtal för \r\nskattebrott. Lagen innehåller också bestämmelser om \r\nförfarandet hos åklagaren samt vissa bestämmelser om \r\nverkställighet. \r\n\r\n3 § När en fråga om skattetillägg ska prövas enligt denna \r\nlag, tillämpas bestämmelserna om skattetillägg i 49 kap. och \r\n51 kap. skatteförfarandelagen (2011:1244). Bestämmelserna i \r\n49 kap. 10 a och 10 b §§ ska dock inte tillämpas.\r\n\r\nEtt beslut om ska

In [154]:
import requests
from bs4 import BeautifulSoup

# Replace <POST_ID> with the actual post ID
url = "https://rkrattsbaser.gov.se/sfst?bet=2022:1559"

# Send a request to the URL
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.text, "html.parser")

# Find the elements in the HTML structure
main_wrapper = soup.find(class_="main wrapper")
content = main_wrapper.find(class_="content")
search_results = content.find(class_="search-results")
search_main = search_results.find(class_="search-main")
search_results_content = search_main.find(class_="search-results-content")

# Initialize the output dictionary
output = {}

# Iterate through the children of 'search-results-content', adding them to the dictionary with index and class name
for index, child in enumerate(search_results_content.children):
    if child.name is not None:
        class_name = " ".join(child["class"])
        key = f"{index}_{class_name}"
        output[key] = child.get_text(strip=True)

# Print the output dictionary
print(output)

{'1_result-inner-box bold': 'SFS-nummer ·\r\n                    2022:1559 ·Visa register', '3_result-inner-box': 'Förordning (2022:1559) om skiktgräns för statlig inkomstskatt för beskattningsåret 2023', '5_result-inner-box': 'Departement:Finansdepartementet', '7_result-inner-box': 'Utfärdad:2022-11-24', '9_result-inner-box': 'Ikraft:2023-01-01', '11_result-inner-box': '1 § Regeringen fastställer enligt 65 kap. 5 § \r\ninkomstskattelagen (1999:1229) skiktgränsen till \r\n598 500 kronor för beskattningsåret 2023.'}


In [150]:
output.keys()

dict_keys(['1_result-inner-box bold', '3_result-inner-box', '5_result-inner-box', '7_result-inner-box', '9_result-inner-box', '11_result-inner-box'])

In [155]:
output

{'1_result-inner-box bold': 'SFS-nummer ·\r\n                    2022:1559 ·Visa register',
 '3_result-inner-box': 'Förordning (2022:1559) om skiktgräns för statlig inkomstskatt för beskattningsåret 2023',
 '5_result-inner-box': 'Departement:Finansdepartementet',
 '7_result-inner-box': 'Utfärdad:2022-11-24',
 '9_result-inner-box': 'Ikraft:2023-01-01',
 '11_result-inner-box': '1 § Regeringen fastställer enligt 65 kap. 5 § \r\ninkomstskattelagen (1999:1229) skiktgränsen till \r\n598 500 kronor för beskattningsåret 2023.'}

In [164]:
!pip install beautifulsoup4
!pip install requests-html

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 6.6 MB/s eta 0:00:0000:0100:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=f14360c51df1bca256eb8ce696e980baec0c059d1eda82921caaf5bb8ac10648
  Stored in directory: /root/.cache/pip/wheels/e4/62/1d/d4d1bc4f33350ff84227f89b258edb552d604138e3739f5c83
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24571 sha256=d1a32ce00e3fdf296b002cb88960e66379ac0348cbfd80391be96f9a3775fe13
  Stored in directory: /root/.cache/pip/wheels/03/d9/92/db136347b5bcba7d271a3c042ce8c9c279e0ecd79173bb0a6e
Successfully built bs4 parse
  Attempting uninstall: websockets
    Found existing 

In [ ]:
# install https://chromedriver.storage.googleapis.com/index.html?path=94.0.4606.41/

In [12]:
import requests
from bs4 import BeautifulSoup

class ContentExtractor:
    
    MAPPING = {
        "5_result-inner-box": "issuer",
        "7_result-inner-box": "issued_date",
        "9_result-inner-box": "in_effect_date",
        "11_result-inner-box": "content",
    }
    
    def __init__(self, post_id):
        
        self.post_id = post_id
        self.url = f"https://rkrattsbaser.gov.se/sfsr?fritext=&upph=false&sort=desc&page=2&post_id={post_id}"
    
    def extract_content(self):
        # Send a request to the URL
        response = requests.get(self.url)

        # Parse the HTML content
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Find the "Visa fulltext" link
        fulltext_element = soup.find("a", href=True, text=lambda text: "Visa fulltext" in text if text else False)

        fulltext_link = fulltext_element["href"]
    
        fulltext_url = f"https://rkrattsbaser.gov.se{fulltext_link}"

        # Send a request to the fulltext URL
        fulltext_response = requests.get(fulltext_url)

        # Parse the HTML content of the fulltext page
        fulltext_soup = BeautifulSoup(fulltext_response.text, "html.parser")

        # Find the elements in the HTML structure
        main_wrapper = fulltext_soup.find(class_="main wrapper")
        content = main_wrapper.find(class_="content")
        search_results = content.find(class_="search-results")
        search_main = search_results.find(class_="search-main")
        search_results_content = search_main.find(class_="search-results-content")

        # Initialize the output dictionary
        output = {}

        # Iterate through the children of 'search-results-content', adding them to the dictionary with index and class name
        for index, child in enumerate(search_results_content.children):
            if child.name is not None:
                class_name = " ".join(child["class"])
                key = f"{index}_{class_name}"
                output[key] = child.get_text(strip=True)
                
        output = {self.MAPPING[k] :v for k, v in output.items() if k in self.MAPPING.keys()}

        return output

# Replace <POST_ID> with the actual post ID
post_id = "2"

# Create a ContentExtractor instance
extractor = ContentExtractor(post_id)

# Extract the content and print the output dictionary
output = extractor.extract_content()
print(output)

{'issuer': 'Departement:Utbildningsdepartementet', 'issued_date': 'Utfärdad:2023-04-05', 'in_effect_date': 'Ikraft:2023-05-10 överg.best.', 'content': '/Träder i kraft I:2023-05-10/\r\nInledande bestämmelser\r\n\r\n1 § Denna förordning innehåller bestämmelser om statsbidrag \r\nför personalkostnader i skolsociala team i grundskolan och \r\ngymnasieskolan.\r\n\r\nMed skolsocialt team avses i förordningen en grupp med \r\npersonal från skolan och socialtjänsten som samverkar för \r\ntrygghet och studiero och för att öka elevernas närvaro i \r\nskolan. Genom deltagande i skolsociala team ges socialtjänsten \r\nförutsättningar att i ett tidigt skede arbeta tillitsskapande \r\noch förebyggande.\r\n\r\nFörordningen är meddelad med stöd av 8 kap. 7 § \r\nregeringsformen.\r\n\r\n2 § Statsbidrag lämnas i mån av tillgång på medel och för ett \r\nkalenderår (bidragsår) i sänder.\r\n\r\n3 § Statsbidrag får inte lämnas för insatser som bidrag lämnas \r\nför på annat sätt. Bidrag får inte heller läm

/opt/conda/lib/python3.10/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


In [1]:
import pandas as pd

In [13]:
outputs = []

for post_id in range(1, 4540):
    
    print(post_id)

    # Create a ContentExtractor instance
    extractor = ContentExtractor(f"{post_id}")

    # Extract the content and print the output dictionary
    output = extractor.extract_content()
    
    outputs.append(output)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699


In [15]:
import pandas as pd

In [16]:
df = pd.DataFrame(outputs)

In [18]:
df.to_pickle("data/laws.pickle")

In [6]:
df = pd.read_pickle("data/laws.pickle")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.3 MB/s eta 0:00:0000:0100:01


In [4]:
import tiktoken
def get_number_tokens(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ").replace("\r", " ")
    enc = tiktoken.encoding_for_model(model)
    return len(enc.encode(text))

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ").replace("\r", " ")
    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

In [7]:
df['tokens'] = df.content.apply(lambda x: get_number_tokens(str(x), model='text-embedding-ada-002'))
# df.to_csv('output/embedded_1k_reviews.csv', index=False)

In [ ]:
df.tokens.sum()

In [9]:
((2782880 / 1000) * 0.004) 

11.13152

In [216]:
df_sub = df.sample(1)

In [218]:
df_sub['ada_embedding'] = df_sub.content.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
# df.to_csv('output/embedded_1k_reviews.csv', index=False)

1 § Den ränta som enligt 19 kap. 47 § socialförsäkringsbalken 
ska betalas på skuld som avser betalningsskyldighet för 
underhållsstöd enligt samma balk ska vara 0,14 procent för år 
2023.


In [219]:
len(df_sub["ada_embedding"].values[0])

1536

In [268]:
text = (df.loc[df.tokens > 40000].content.values[0])

In [357]:
def split_text(text, chunk_size, overlap):
    chunks = []
    start = 0
    end = chunk_size
    while start < len(text):
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap
        end += chunk_size - overlap
    return chunks

chunks = split_text(text, chunk_size=2000, overlap=200)

In [358]:
df = pd.DataFrame(outputs)
df['content'] = df['content'].apply(lambda x: split_text(str(x), 15000, 200))
df = df.explode('content')

In [359]:
df['tokens'] = df.content.apply(lambda x: get_number_tokens(str(x), model='text-embedding-ada-002'))

In [360]:
df["tokens"].max()

5403

In [307]:
import re
split_pattern = r"\n\d+ kap"

items = re.split(split_pattern, text)[1:]

for item in items:
    print(len(item))

4847
7182
2437
18576
137
10838
11518
2902
7611
3625
9075
12948
1389
15775
3294
14817
2795
3229
1103
1429


In [290]:
print(text)

1 kap. Gemensamma bestämmelser

1 § Denna förordning har meddelats med stöd av 

- 2 § lagen (1994:1708) om EG:s förordningar om strukturstöd 
och om stöd till utveckling av landsbygden och 4 § lagen 
(1994:1710) om EG:s förordningar om jordbruksprodukter i fråga 
om 11 kap. 3-6 §§, 

- 3 a § lagen om EU:s förordningar om strukturstöd och om stöd 
till utveckling av landsbygden i fråga om 5 kap. 14 §,

- 3 § lagen om EG:s förordningar om jordbruksprodukter i fråga 
om 3 kap. 33 §§ och 6 kap. 17 §, 

- 6 § lagen om EU:s förordningar om strukturstöd och om stöd 
till utveckling av landsbygden och 10 § lagen om EG:s 
förordningar om jordbruksprodukter i fråga om 13 kap. 6 §, 

- 1 § lagen (2015:266) om lokala aktionsgrupper i fråga om 
3 kap. 11-16 §§, och 

- 8 kap. 7 § regeringsformen i fråga om övriga bestämmelser. 

2 § Denna förordning kompletterar 

1. Europaparlamentets och rådets förordning (EU) 2021/2115 av 
den 2 december 2021 om fastställande av regler om stöd för de 
strategis

'. Kompensationsstöd för bergsområden, områden med \r\nnaturliga begränsningar och områden med särskilda \r\nbegränsningar\r\n\r\n1 § Stöd enligt detta kapitel får lämnas till jordbruksföretag \r\ni områden med naturliga eller andra områdesspecifika \r\nbegränsningar enligt artikel 71 i Europaparlamentets och \r\nrådets förordning (EU) 2021/2115.\r\n\r\n2 § Stöd får lämnas för odling av grovfodergrödor, \r\nväxtodlingsgrödor och extensiva grödor med de belopp och till \r\nde områden som framgår av bilagorna 5 och 6.\r\n\r\n3 § Stöd lämnas inte för mark som huvudsakligen används för \r\nnågot annat ändamål än jordbruk eller för mark som används på \r\nett sätt som är oförenligt med stödets syfte enligt bilaga IX \r\ntill kommissionens genomförandeförordning (EU) 2022/128.\r\n\r\nFörutsättningar för stöd \r\n\r\n4 § Ett jordbruksskifte ska omfatta en sammanhållen areal om \r\nminst 0,1 hektar för att berättiga till stöd. För arealer som \r\nockså ger rätt till stöd för skyddszoner mot va

In [305]:
len(items)

20

In [306]:
text

'1 kap. Gemensamma bestämmelser\r\n\r\n1 § Denna förordning har meddelats med stöd av \r\n\r\n- 2 § lagen (1994:1708) om EG:s förordningar om strukturstöd \r\noch om stöd till utveckling av landsbygden och 4 § lagen \r\n(1994:1710) om EG:s förordningar om jordbruksprodukter i fråga \r\nom 11 kap. 3-6 §§, \r\n\r\n- 3 a § lagen om EU:s förordningar om strukturstöd och om stöd \r\ntill utveckling av landsbygden i fråga om 5 kap. 14 §,\r\n\r\n- 3 § lagen om EG:s förordningar om jordbruksprodukter i fråga \r\nom 3 kap. 33 §§ och 6 kap. 17 §, \r\n\r\n- 6 § lagen om EU:s förordningar om strukturstöd och om stöd \r\ntill utveckling av landsbygden och 10 § lagen om EG:s \r\nförordningar om jordbruksprodukter i fråga om 13 kap. 6 §, \r\n\r\n- 1 § lagen (2015:266) om lokala aktionsgrupper i fråga om \r\n3 kap. 11-16 §§, och \r\n\r\n- 8 kap. 7 § regeringsformen i fråga om övriga bestämmelser. \r\n\r\n2 § Denna förordning kompletterar \r\n\r\n1. Europaparlamentets och rådets förordning (EU) 2021/21